<a href="https://colab.research.google.com/github/ranjitben10/Rag_examples/blob/main/Task_astria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community pypdf google-generativeai langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [2]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro")

In [48]:
!pip install langchain_ollama

In [51]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="llama3.1")


In [5]:
%pip install --upgrade --quiet  docx2txt

  Preparing metadata (setup.py) ... done


In [8]:
from langchain.document_loaders import PyPDFLoader, DataFrameLoader
import pandas as pd
import google.generativeai as genai
GOOGLE_API_KEY = "AIzaSyAziDIq4oAKaZZV45Jg-6aub8sNEn"
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-pro")

In [9]:
from langchain_community.document_loaders import Docx2txtLoader

In [11]:
%pip install --upgrade --quiet langchain-community unstructured openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.4 MB/s eta 0:00:00


In [12]:
from langchain_community.document_loaders import UnstructuredExcelLoader

In [14]:


# Load PDF document
pdf_loader = PyPDFLoader("/content/RFP1.pdf")
pdf_documents = pdf_loader.load()

# Load Excel document
excel_loader = UnstructuredExcelLoader("/content/RFP2.xlsx", mode="elements")
excel_documents = excel_loader.load()

# Load Word document
docx_loader = Docx2txtLoader("/content/RFP3.docx")
docx_documents = docx_loader.load()

# Combine all documents into a single list
all_documents = pdf_documents + excel_documents + docx_documents

# Add metadata to differentiate sources (optional but helpful)
for doc in pdf_documents:
    doc.metadata["source"] = "pdf"
for doc in excel_documents:
    doc.metadata["source"] = "excel"
for doc in docx_documents:
    doc.metadata["source"] = "docx"


In [19]:
len(all_documents)

8

In [71]:
prompt_text = """
You are an Information Extractor tasked with identifying specific event-related details from the provided document content. Carefully interpret and extract relevant information based on context, and include each field as part of your response if it is available.

### Instructions:
1. For each event, identify details such as date, day, start time, end time, function type, setup style, people count, and any additional comments.
2. If any information is missing in the document, simply omit it in the response.
3. Ensure that each response contains only the extracted details for each event, listing each available piece of information without enforcing empty fields.

### Desired Response Format:
- Provide each event as a collection of extracted fields with details such as:
  - **Date**: Extracted as MM/DD/YYYY if available.
  - **Day**: The day of the week associated with the event, if mentioned.
  - **Repeat Count**: Number of repetitions if specified.
  - **Start Time**: Start time for the event.
  - **End Time**: End time for the event.
  - **Function Type**: Type of function or activity, like "Office", "Storage", etc.
  - **Setup Style**: Setup style or layout for the event.
  - **People Count**: Expected number of attendees if mentioned.
  - **Comments**: Additional notes or relevant details.

Document Content:
{context}
"""


In [65]:
# prompt_text="""
# You are an advanced Information Extractor tasked with identifying specific event-related details from the provided document content. Carefully interpret and map each piece of information to the correct field based on context, prioritizing only relevant information for each key.

# ### Instructions:
# 1. **Identify Key Information**: For each event, identify and classify details such as date, day, start time, end time, function type, setup style, people count, and additional comments.
#    - **Focus on Recognizable Cues**: Use language cues in the document, such as "day," "date," "people," or "type," to help categorize information accurately.
# 2. **Map to Fields Dynamically**: Each field should contain only the information relevant to it, avoiding unnecessary details or placeholder text (like "N/A") for missing information.
#    - Omit fields where the document provides no relevant content.
# 3. **Include Contextual Keywords**: Pay attention to contextual words that indicate function type (e.g., "meeting," "conference"), setup style (e.g., "banquet," "theater"), or attendee count.
# 4. **Avoid Redundant Fields**: If details are repeated or implied by other fields (e.g., "arrival day" suggests "day" as "Sunday"), consolidate these logically within one field.

# ### Desired Response Format:
# - For each event, include only relevant extracted fields with details such as:
#   - **Date**: Provide as MM/DD/YYYY if available; otherwise, use descriptive timing like "Mid-February" where precise dates are absent.
#   - **Day**: The specific day of the week (e.g., "Monday") or day range (e.g., "Saturday - Wednesday") as mentioned.
#   - **Repeat Count**: Include only if the document specifies a repeat count or recurrence.
#   - **Start Time** and **End Time**: Times provided for the start and end of the event.
#   - **Function Type**: Type of function (e.g., "Annual Sales Meeting," "Vendor Tradeshow") if provided.
#   - **Setup Style**: Event setup style (e.g., "classroom," "banquet").
#   - **People Count**: Expected number of attendees; map any mention of numbers related to people here.
#   - **Comments**: Additional details, instructions, or notes relevant to the event setup or requirements.

# For each event, present only the fields containing relevant extracted information.

# Document Content:
# {context}

# """

In [67]:
# prompt_text = """
# You are an Information Extractor specialized in event planning details. Your task is to analyze the provided document content and accurately categorize details into specific fields, using both keywords and contextual understanding to guide your choices. Focus on identifying the most relevant information for each field, avoiding any placeholders or redundant entries.

# ### Instructions for Field Extraction:
# 1. **Identify Relevant Information**: For each event, determine key details such as date, day, start time, end time, function type, setup style, people count, and comments.
#    - **Use Content Cues**: Identify phrases, keywords, or patterns that indicate specific fields (e.g., "Saturday - Wednesday" as **Day**, "2800 people" as **People Count**).
#    - **Filter Out Non-relevant Data**: Only include information directly related to event details; ignore unrelated text.

# 2. **Map to Appropriate Fields**:
#    - Include each field only if it is present in the document.
#    - Avoid placeholders (like "N/A") for missing data.

# 3. **Field-specific Guidance**:
#    - **Date**: Look for exact dates (e.g., "02/15/2013") or timeframes (e.g., "Mid-February") and provide as **Date**.
#    - **Day**: Extract the day or range of days if specified (e.g., "Monday" or "Saturday - Wednesday").
#    - **Repeat Count**: Include only if a repetition or recurrence is mentioned (e.g., "repeats weekly").
#    - **Start Time / End Time**: Include event start and end times if they are given in hours or time expressions.
#    - **Function Type**: Identify event types like "Sales Meeting," "Tradeshow," or similar descriptors that indicate the purpose or nature of the event.
#    - **Setup Style**: Note the event’s layout setup, such as "banquet," "classroom," or "theater style" if mentioned.
#    - **People Count**: Map any specific number of attendees to **People Count** (e.g., "2800 people").
#    - **Comments**: Include any special notes, instructions, or specific requirements related to the event setup, such as "complimentary internet at Registration Desk."

# ### Response Format:
# - For each event, list only the fields that contain relevant information in the following format:
#   - **Date**: [Event date, if provided]
#   - **Day**: [Day of the event, if provided]
#   - **Repeat Count**: [Recurrence details, if mentioned]
#   - **Start Time**: [Event start time, if available]
#   - **End Time**: [Event end time, if available]
#   - **Function Type**: [Event function type, if provided]
#   - **Setup Style**: [Event setup style, if available]
#   - **People Count**: [Attendee count, if available]
#   - **Comments**: [Additional notes or requirements, if provided]

# Document Content:
# {context}

# """

In [60]:
import json
import re

# Sample output string (replace this with your actual output)
response = """
- **Date**: N/A
- **Day**: Sunday
- **Repeat Count**: N/A
- **Start Time**: N/A
- **End Time**: N/A
- **Function Type**: Annual Sales Meeting
- **Setup Style**: N/A
- **People Count**: 1890
- **Comments**: Main Arrivals

- **Date**: N/A
- **Day**: Monday
- **Repeat Count**: N/A
- **Start Time**: N/A
- **End Time**: N/A
- **Function Type**: Annual Sales Meeting
- **Setup Style**: N/A
- **People Count**: 1890
- **Comments**: N/A
"""

# Split the response into individual event strings
event_strings = re.split(r'(?=- \*\*Date\*\*:)', response.strip())

# Create a list to hold event dictionaries
events = []

# Process each event string to extract details
for event_str in event_strings:
    event_dict = {}

    # Extract each field using regex
    for line in event_str.splitlines():
        if ": " in line:
            key, value = line.split(": ", 1)  # Split on first occurrence
            # Clean up the key and value
            key = key.strip().replace("- **", "").replace("**", "")
            value = value.strip()
            event_dict[key] = value

    # Append the event dictionary to the events list if it's not empty
    if event_dict:
        events.append(event_dict)

# Write the events list to a JSON file
with open('events_output.json', 'w') as json_file:
    json.dump({"events": events}, json_file, indent=4)

print("Events have been written to events_output.json")


Events have been written to events_output.json


In [61]:
events

[{'Date': 'N/A',
  'Day': 'Sunday',
  'Repeat Count': 'N/A',
  'Start Time': 'N/A',
  'End Time': 'N/A',
  'Function Type': 'Annual Sales Meeting',
  'Setup Style': 'N/A',
  'People Count': '1890',
  'Comments': 'Main Arrivals'},
 {'Date': 'N/A',
  'Day': 'Monday',
  'Repeat Count': 'N/A',
  'Start Time': 'N/A',
  'End Time': 'N/A',
  'Function Type': 'Annual Sales Meeting',
  'Setup Style': 'N/A',
  'People Count': '1890',
  'Comments': 'N/A'}]

In [72]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import json,re
# Set up the prompt template and LLMChain
prompt_template = PromptTemplate(input_variables=["context"], template=prompt_text)
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
events=[]
res=""
# Run the chain for each document individually
for doc in all_documents:
    document_content = doc.page_content
    #print(document_content)
    output = llm_chain.run({"context": document_content})
    events.append({"content": output})
    res+=output

# Split the response into individual event strings
event_strings = re.split(r'(?=- \*\*Date\*\*:)', res.strip())

# Create a list to hold event dictionaries
events = []

# Process each event string to extract details
for event_str in event_strings:
    event_dict = {}

    # Extract each field using regex
    for line in event_str.splitlines():
        if ": " in line:
            key, value = line.split(": ", 1)  # Split on first occurrence
            # Clean up the key and value
            key = key.strip().replace("- **", "").replace("**", "")
            value = value.strip()
            event_dict[key] = value

    # Append the event dictionary to the events list if it's not empty
    if event_dict:
        events.append(event_dict)

# Write the events list to a JSON file
with open('events_output.json', 'w') as json_file:
    json.dump({"events": events}, json_file, indent=4)

print("Events have been written to events_output.json")


Events have been written to events_output.json


In [64]:
events[10]

{'Date': '07/30/2023 - 08/02/2023',
 'Day': 'Friday - Monday',
 'Repeat Count': '3',
 'Start Time': '08:00 AM',
 'End Time': '24 hour - Wed. noon',
 'Function Type': 'General Session / Ballroom',
 'Setup Style': 'Rounds',
 'People Count': '2,500',
 'Comments': '60,000 sf.'}